In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, accuracy_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/new drive/data mining/Housing.csv')
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [ ]:
# Handle missing values (if any)
df.dropna(inplace=True)

In [ ]:
# Define features and target variable
X = df.drop(columns=["price"])
y = df["price"]

In [ ]:
# Convert price into a binary classification problem for logistic regression (high price vs low price)
median_price = np.median(y)
y_class = (y > median_price).astype(int)

In [ ]:
# Identify categorical and numerical columns
categorical_features = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea", "furnishingstatus"]
numerical_features = ["area", "bedrooms", "bathrooms", "stories", "parking"]


In [ ]:
# Preprocessing: One-Hot Encoding for categorical data, Standard Scaling for numerical data
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(drop='first'), categorical_features)
])

In [ ]:
# Create a pipeline with preprocessing and Linear Regression model
mlr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Train the Multiple Linear Regression model
mlr_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['area', 'bedrooms',
                                                   'bathrooms', 'stories',
                                                   'parking']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['mainroad', 'guestroom',
                                                   'basement',
                                                   'hotwaterheating',
                                                   'airconditioning',
                                                   'prefarea',
                                                   'furnishingstatus'])])),
                ('regressor', LinearRegression())])

In [ ]:
# Predict house prices
y_pred = mlr_pipeline.predict(X_test)

In [ ]:
# Evaluate the Multiple Linear Regression model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

In [ ]:
print("Multiple Linear Regression Model Performance:")
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")
print(f"R-squared (Accuracy): {r2}")

Multiple Linear Regression Model Performance:
Mean Absolute Error: 920392.9397607958
Root Mean Squared Error: 1234106.749637685
R-squared (Accuracy): 0.6463350878895876


In [ ]:
# Logistic Regression Model
logistic_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

In [ ]:
# Split the dataset into training and testing sets for logistic regression
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(X, y_class, test_size=0.2, random_state=42)

In [ ]:
# Train the Logistic Regression model
logistic_pipeline.fit(X_train_class, y_train_class)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['area', 'bedrooms',
                                                   'bathrooms', 'stories',
                                                   'parking']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['mainroad', 'guestroom',
                                                   'basement',
                                                   'hotwaterheating',
                                                   'airconditioning',
                                                   'prefarea',
                                                   'furnishingstatus'])])),
                ('classifier', LogisticRegression())])

In [ ]:
#predict house price
y_pred_class = logistic_pipeline.predict(X_test_class)

In [ ]:
# Evaluate the Logistic Regression model
accuracy = accuracy_score(y_test_class, y_pred_class)
print(f"Accuracy: {accuracy}")

Accuracy: 0.8440366972477065


In [ ]:
# Function to predict price for a new house
def predict_price(new_house):
    new_house_df = pd.DataFrame([new_house])
    return mlr_pipeline.predict(new_house_df)[0]

In [ ]:
# Function to classify if the price is high or low
def classify_price(new_house):
    new_house_df = pd.DataFrame([new_house])
    return logistic_pipeline.predict(new_house_df)[0]

In [ ]:
# Example usage:
example_house = {
    "area": 7500,
    "bedrooms": 3,
    "bathrooms": 2,
    "stories": 2,
    "mainroad": "yes",
    "guestroom": "no",
    "basement": "no",
    "hotwaterheating": "no",
    "airconditioning": "yes",
    "parking": 2,
    "prefarea": "yes",
    "furnishingstatus": "furnished"
}

In [ ]:
predicted_price = predict_price(example_house)
predicted_class = classify_price(example_house)
print(f"Predicted Price: {predicted_price}")
print(f"Predicted Price Category (1 = High, 0 = Low): {predicted_class}")

Predicted Price: 7476563.679746402
Predicted Price Category (1 = High, 0 = Low): 1
